In [1]:
@file:DependsOn("io.ktor:ktor-client-core:2.3.12")
@file:DependsOn("io.ktor:ktor-client-cio:2.3.12")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.3.12")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.3.12")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe

import ro.jf.funds.client.notebook.*

val client = FundsClient()
val user = client.ensureUserExists("Johann-16.0")
user

UserTO(id=9ab40120-eeae-4744-93b3-16ef13dcd0aa, username=Johann-16.0)

In [2]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk

val ACCOUNTS_YAML_FILE = "../../data/provision/accounts.yaml"
val FUNDS_YAML_FILE = "../../data/provision/funds.yaml"
val IMPORT_CONFIGURATION_JSON_FILE = "../../data/provision/import-configuration.json"
val IMPORT_CONFIGURATION_YAML_FILE = "../../data/provision/import-configuration.yaml"

val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()


In [3]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking
import com.charleskorn.kaml.Yaml
import kotlinx.serialization.builtins.ListSerializer

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(user.id).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = File(ACCOUNTS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateAccountTO.serializer()), it) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(user.id, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=e4ebe379-ab85-4f09-a4a8-64f193c467f8, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=d242a936-29d2-4d4a-bb9c-97f4a7e1d6a9, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=aae64847-4112-4997-a65d-b0271f1dd9a7, name=ING RON, unit=Currency(value=RON))
AccountTO(id=84947bdc-3e8a-4e64-b291-1fb7bc63c1c2, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=4ae17927-9f0a-4598-811c-39aed9b2dd47, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=58385a02-854b-4d9c-94c4-480d9149bb6b, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=5d28c889-1958-43f0-8758-b213573e719a, name=Other, unit=Currency(value=RON))
AccountTO(id=f0612d54-d107-4fdb-90b6-8e5806358ce3, name=BT RON, unit=Currency(value=RON))
AccountTO(id=28fae71a-c01d-49af-a7e8-a695b70970c7, name=BT Economy RON, unit=Currency(value=RON))

In [4]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    val existingFunds = fundSdk.listFunds(user.id).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = File(FUNDS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateFundTO.serializer()), it) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(user.id, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=b607504a-72b4-429e-b83a-9b93ce371487, name=Expenses)
FundTO(id=3862b91a-fa7a-4275-9405-9ed1ececce21, name=Work Income)
FundTO(id=a7be60c0-5d0d-4709-a0c0-4d2f36f31a17, name=Savings)
FundTO(id=f884e860-d94f-4f4d-8e0f-334fbc2a73ba, name=Gifts)

In [5]:
import ro.jf.funds.importer.api.model.*
import kotlinx.serialization.json.Json

val dataConfigurationRawYaml: String = File(IMPORT_CONFIGURATION_YAML_FILE).readText()
val importConfiguration = Yaml.default.decodeFromString(ImportConfigurationTO.serializer(), dataConfigurationRawYaml)

In [6]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

val csvFiles =
    listOf(File("../../data/wallet/2019"), File("../../data/wallet/2020"))
        .flatMap { it.listFiles()?.toList() ?: error("no files found") }
runBlocking {
    var importTask = importSdk.import(user.id, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(user.id, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=5d6e5c23-cf47-4d1d-bf9c-7daaf9db87f4, status=COMPLETED, reason=null)